In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('/content/diabetes.csv')

In [ ]:
df.head()

In [ ]:
X=df.iloc[:,:-1] ## independent features
y=df.iloc[:,-1] ## dependent features

In [ ]:
scalar = StandardScaler()
X = scalar.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.layers import  Dense,Dropout
from keras import Sequential


In [ ]:
model = Sequential()
model.add(Dense(32,activation = 'relu', input_dim = 8))
model.add(Dense(1,activation='sigmoid',))

model.compile(optimizer = 'adam',loss = 'binary_crossentropy',  metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,y_train, batch_size= 32,epochs= 100,validation_data = (X_test,y_test))

In [ ]:
# how to select the appropiate optimizer
# no. of nodes/ neuron in a layer
#how to select the number of layers
# all in one model

In [ ]:
pip install keras-tuner --upgrade


In [ ]:
import keras_tuner as kt
from keras_tuner import RandomSearch

how to get the best optimizer

In [ ]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32,activation = 'relu', input_dim = 8))
  model.add(Dense(1,activation='sigmoid',))
  optimizers = hp.Choice('optimizers', values =['adam','sgd', 'rmsprop', 'adadelta'] )
  model.compile(optimizer = optimizers,loss = 'binary_crossentropy',  metrics = ['accuracy'])
  return model


In [ ]:
# Define the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,  # Run each trial once
    directory='my_tuning_dir',
    project_name='optimizer_tuning'
)

In [ ]:
tuner.search(X_train, y_train, epochs=5,validation_data=(X_test,y_test))

In [ ]:
print("Results for all trials:")
for trial in tuner.oracle.get_best_trials(num_trials=len(tuner.oracle.trials)):
    print(f"Trial ID: {trial.trial_id}")
    print(f"Optimizer: {trial.hyperparameters.get('optimizers')}")

    # Access the best validation accuracy value
    val_accuracy = trial.metrics.get_best_value('val_accuracy')
    print(f"Validation Accuracy: {val_accuracy}")
    print("---------------------------------------------------")

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Optimizer: {best_hps.get('optimizers')}")


In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,y_train, batch_size= 32,epochs= 100,initial_epoch=6,validation_data = (X_test,y_test))

In [ ]:
model = Sequential()
model.add(Dense(32,activation = 'relu', input_dim = 8))
model.add(Dense(1,activation='sigmoid',))

model.compile(optimizer = 'rmsprop',loss = 'binary_crossentropy',  metrics = ['accuracy'])

In [ ]:
model.fit(X_train,y_train, batch_size= 32,epochs= 100,initial_epoch=6,validation_data = (X_test,y_test))

how to find best optimum number of nodes per layer

In [ ]:
from keras_tuner import RandomSearch
from tensorflow.keras import layers, models, optimizers
import tensorflow as tf

# Define a function to build the model, with units as a tunable hyperparameter
def build_model(hp):
    model = models.Sequential()

    # Define a tunable number of units for the hidden layer (between 32 and 256)
    model.add(layers.Dense(units=hp.Int('units', min_value=8, max_value=128),
                           activation='relu', input_shape=(X_train.shape[1],)))

    # Output layer for binary classification
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile the model with RMSprop optimizer (fixed)
    model.compile(optimizer=optimizers.RMSprop(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Set up the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',  # Optimize validation accuracy
    max_trials=5,               # Number of trials to run
    executions_per_trial=1,     # Number of executions per trial
    directory='my_tuning_dir',  # Directory for storing tuning results
    project_name='best_units_tuning',  # Name of the project
    overwrite=True             # Overwrite previous results
)

# Debugging: Print the shapes of the training and validation data
print(f"Training data shape: {X_train.shape}, {y_train.shape}")
print(f"Validation data shape: {X_test.shape}, {y_test.shape}")

# Run the tuner search to find the best number of units
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test), verbose=1)

# Get the best hyperparameters after the search
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print all the best hyperparameters
print("Best Hyperparameters:", best_hps.values)

# Extract the best 'units' from the best hyperparameters
# Extract the best 'units' from the best hyperparameters using the correct method
best_units = best_hps['units']

# If 'units' is found, print it
print(f"Best Number of Units for Hidden Layer: {best_units}")


# If 'units' is found, print it
if best_units:
    print(f"Best Number of Units for Hidden Layer: {best_units}")
else:
    print("No 'units' found in best hyperparameters.")

# Build the best model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hps)

# Train the best model further
history = best_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Print a summary of the training process
print("\nTraining Summary:")
print("History:", history.history)


In [ ]:
best_model.fit(X_train, y_train, batch_size= 32,epochs=100,initial_epoch= 6, validation_data=(X_test, y_test))

In [ ]:
# Extract the metrics from the history object
train_loss = history.history['loss']
train_accuracy = history.history['accuracy']
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

# Calculate the average for each metric
avg_train_loss = sum(train_loss) / len(train_loss)
avg_train_accuracy = sum(train_accuracy) / len(train_accuracy)
avg_val_loss = sum(val_loss) / len(val_loss)
avg_val_accuracy = sum(val_accuracy) / len(val_accuracy)

# Print the average values
print(f"Average Training Loss: {avg_train_loss}")
print(f"Average Training Accuracy: {avg_train_accuracy}")
print(f"Average Validation Loss: {avg_val_loss}")
print(f"Average Validation Accuracy: {avg_val_accuracy}")

In [ ]:
# Extract the metrics from the history object
train_loss = history.history['loss']
train_accuracy = history.history['accuracy']
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

# Calculate the average for each metric
avg_train_loss = sum(train_loss) / len(train_loss)
avg_train_accuracy = sum(train_accuracy) / len(train_accuracy)
avg_val_loss = sum(val_loss) / len(val_loss)
avg_val_accuracy = sum(val_accuracy) / len(val_accuracy)

# Calculate the best (minimum for loss and validation loss, maximum for accuracy and validation accuracy)
best_train_loss = min(train_loss)
best_train_accuracy = max(train_accuracy)
best_val_loss = min(val_loss)
best_val_accuracy = max(val_accuracy)

# Print the results
print(f"Average Training Loss: {avg_train_loss}")
print(f"Average Training Accuracy: {avg_train_accuracy}")
print(f"Average Validation Loss: {avg_val_loss}")
print(f"Average Validation Accuracy: {avg_val_accuracy}")

print(f"\nBest Training Loss: {best_train_loss}")
print(f"Best Training Accuracy: {best_train_accuracy}")
print(f"Best Validation Loss: {best_val_loss}")
print(f"Best Validation Accuracy: {best_val_accuracy}")


In [ ]:
# Build the best model using the hyperparameters found by the tuner
best_model = tuner.hypermodel.build(best_hps)

# Train the model with the best units
history = best_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Extract the metrics from the history object
train_loss = history.history['loss']
train_accuracy = history.history['accuracy']
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

# Calculate the average for each metric
avg_train_loss = sum(train_loss) / len(train_loss)
avg_train_accuracy = sum(train_accuracy) / len(train_accuracy)
avg_val_loss = sum(val_loss) / len(val_loss)
avg_val_accuracy = sum(val_accuracy) / len(val_accuracy)

# Calculate the best (minimum for loss and validation loss, maximum for accuracy and validation accuracy)
best_train_loss = min(train_loss)
best_train_accuracy = max(train_accuracy)
best_val_loss = min(val_loss)
best_val_accuracy = max(val_accuracy)

# Print the results
print(f"\nAverage Training Loss: {avg_train_loss}")
print(f"Average Training Accuracy: {avg_train_accuracy}")
print(f"Average Validation Loss: {avg_val_loss}")
print(f"Average Validation Accuracy: {avg_val_accuracy}")

print(f"\nBest Training Loss: {best_train_loss}")
print(f"Best Training Accuracy: {best_train_accuracy}")
print(f"Best Validation Loss: {best_val_loss}")
print(f"Best Validation Accuracy: {best_val_accuracy}")


now we will see how to select the number of layers using keras tuner (hyperparameter tunning)...

In [ ]:
# to find num of layers
def build_model(hp):
  model = Sequential()
  model.add(Dense(63,activation = 'relu', input_dim = 8))
  for i in range(hp.Int('num_layer',min_value = 1, max_value=10,)):
    model.add(Dense(63, activation= 'relu'))
  model.add(Dense(1,activation = 'sigmoid'))

  model.compile(optimizer = 'rmsprop',loss = 'binary_crossentropy', metrics = ['accuracy'])

  return model



In [ ]:
tuner = RandomSearch(build_model,
                     objective= 'val_accuracy',
                     max_trials = 5,
                     directory = 'my_tuning_dir',
                     project_name = 'num_layers',
                     overwrite=True)

In [ ]:
tuner.search(X_train,y_train, epochs = 5, validation_data=(X_test, y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(X_train,y_train, epochs=100, initial_epoch = 6, validation_data = (X_test,y_test))

In [ ]:
def build_model(hp):
  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layer',min_value = 1, max_value=10)):

    if counter == 0:
      model.add(
          Dense(
              units=hp.Int('units' + str(i), min_value=8, max_value=128, step =8),
              activation =hp.Choice('activation' + str(i), values = ['relu','sigmoid','tanh']),
              input_dim=8
              )
      )
    else:
      model.add(
          Dense(
              units=hp.Int('units' + str(i), min_value=8, max_value=128, step =8),
              activation =hp.Choice('activation' + str(i), values = ['relu','sigmoid','tanh']),

              )
      )
    counter += 1
  model.add(Dense(1,activation = 'sigmoid'))

  optimizers = hp.Choice('optimizers', values =['adam','sgd', 'rmsprop', 'adadelta'] )
  model.compile(optimizer = optimizers,loss = 'binary_crossentropy',  metrics = ['accuracy'])


  return model






In [ ]:
tuner = RandomSearch(build_model,
                     objective= 'val_accuracy',
                     max_trials = 5,
                     directory = 'my_tuning_dir',
                     project_name = 'all of all',
                     overwrite=True)

In [ ]:
tuner.search(X_train,y_train, epochs = 5, validation_data=(X_test, y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
model =  tuner.get_best_models(num_models =1)[0]

In [ ]:
model.fit(X_train,y_train,epochs =200, initial_epoch = 6, validation_data = (X_test,y_test))

Since the validation accuracy decreasing so we use the drop out layer in this .

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras_tuner import HyperParameters

def build_model(hp):
    model = Sequential()

    # Initialize counter
    counter = 0

    # Loop to create layers based on hyperparameters
    for i in range(hp.Int('num_layer', min_value=1, max_value=10)):
        if counter == 0:  # First layer has input_dim
            model.add(
                Dense(
                    units=hp.Int('units_' + str(i), min_value=8, max_value=128, step=8),
                    activation=hp.Choice('activation_' + str(i), values=['relu', 'sigmoid', 'tanh']),
                    input_dim=8
                )
            )
        else:
            model.add(
                Dense(
                    units=hp.Int('units_' + str(i), min_value=8, max_value=128, step=8),
                    activation=hp.Choice('activation_' + str(i), values=['relu', 'sigmoid', 'tanh'])
                )
            )

        # Adding Dropout after each Dense layer
        model.add(
            Dropout(
                hp.Choice("dropout_" + str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
            )
        )

        # Increment the counter after adding the layer
        counter += 1

    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    # Hyperparameter for optimizer
    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta'])

    # Compile model
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [ ]:
tuner = RandomSearch(build_model,
                     objective= 'val_accuracy',
                     max_trials = 5,
                     directory = 'my_tuning_dir',
                     project_name = 'all of all-final',
                     overwrite=True)

In [ ]:
tuner.search(X_train,y_train, epochs = 5, validation_data=(X_test, y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
model =  tuner.get_best_models(num_models =1)[0]

In [ ]:
model.fit(X_train,y_train,epochs =200, initial_epoch = 6, validation_data = (X_test,y_test))

In [ ]:
import matplotlib.pyplot as plt

# Extract accuracy and validation accuracy from the history object
history_dict = history.history
train_acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']

# Plot training and validation accuracy
plt.plot(train_acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')

# Add labels and title
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy')
plt.legend()

# Show the plot
plt.show()
